# **Accidental Drug Related Deaths**


# **1. Introduction**

 I want to explore the relationships between age,race,sex and drug use overdose.
 
 I want build pattern:
* some races, sexes, age or drugs of choice impact of death
* race, sex, or age are determine the likelihood of someone overdosing on a specific drug

Necessary to predict what risk of mortality applies to each of the groups under consideration, using certain types of drugs

This will help identify the most dangerous types of drugs

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/accidental-drug/Accidental_Drug.csv')

Let's see what data we have:

In [ ]:
df.info(memory_usage='deep')

In [ ]:
df = df.drop(['DeathCityGeo','ResidenceCityGeo','InjuryCityGeo','DeathCity','DeathCounty'],axis = 1)
df['Age'].fillna(df['Age'].median(), inplace = True)
df.head()

# 2. Check and correct data
# Сreating a data visualization 

Let's accept the statement that Value 'DateReported' was not end by death

In [ ]:
df['DateType'].value_counts()

In [ ]:
df['Death'] = df.DateType.map(lambda x : 0 if x == 'DateofDeath' else 1)

In [ ]:
#Let’s see how many of each Race is in our data set
df.Race.fillna('Unknown', inplace=True)
df['Race'].value_counts()

On this diagram we can see, that most victoms are white:

In [ ]:
df['Race'].value_counts().plot.bar()

On this diagram we can see, that most victoms are Male


In [ ]:
df['Sex'].value_counts()

In [ ]:
#Let’s see how many of each sex is in our data set
df['Sex'].value_counts().plot.bar()

In [ ]:
df['Female'] = df.Sex.map(lambda x : 1 if x == 'Female' else 0)
df['Male'] = df.Sex.map(lambda x : 1 if x == 'Male' else 0)

In [ ]:
#Let’s see types of drugs
df.columns[18:36]

In [ ]:
for drug in df.columns[18:36]:
    df[drug] = df[drug].map(lambda x : 1 if x == 'Y' else 0)   
df['NDrugs'] = df.apply(lambda x: x[18:36].sum(), axis='columns')

If man use differnt drugs, we'll count it and crate new column - NDrugs

In [ ]:
Drugs = df.groupby('NDrugs').ID.count()
Drugs.plot.bar(x=Drugs.index, y=Drugs.values)

We can see, that most addict are white male, used 1-3 types of drugs

In [ ]:
list_ = []
list_of_drugs = df.columns[18:36]
for i in list_of_drugs:
    list_.append(df[i].sum())
Drugs_df = pd.DataFrame(list_,list_of_drugs)
Drugs_df.sort_values(by=[0],ascending=False).plot.bar()

Most used drugs which lead to overdose : Heroin, Opiod, Fentanyl

# 3.ML
Let's go to the search for values that affect mortality


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

Removing insignificant columns

In [ ]:
df_source = df
df_output = df['Death']
df_plt = df.drop(['ID','Date','DateType','OtherSignifican', 'Sex','Race','ResidenceCity','ResidenceCounty','ResidenceState','Location','LocationifOther','DescriptionofInjury','InjuryPlace','InjuryCity','InjuryCounty','InjuryState','COD','OtherSignifican'],axis = 1) 
df = df.drop(['Death','ID','Date','DateType','OtherSignifican', 'Sex','Race','ResidenceCity','ResidenceCounty','ResidenceState','Location','LocationifOther','DescriptionofInjury','InjuryPlace','InjuryCity','InjuryCounty','InjuryState','COD','OtherSignifican'],axis = 1)

Create test dataset, to fill and chek our dataset

In [ ]:
inputs_train, inputs_test, expected_output_train, expected_output_test = train_test_split(df, df_output, test_size = 0.33, random_state = 42)

We can use RandomForest ML
Random forest is a classification algorithm consisting of many decision trees. It uses bagging and feature randomness when building each individual tree to try to create an uncorrelated forest whose prediction is more accurate than that of any individual tree.

In [ ]:
rf = RandomForestClassifier (n_estimators=100)

In [ ]:
rf.fit(inputs_train, expected_output_train)
accuracy = rf.score(inputs_test, expected_output_test)
print("Accuracy = {}%".format(accuracy * 100))

We trained our model and got an accuracy of 0.6

Let's see what parameters affect mortality

In [ ]:
f_={}
for feature, importance in zip(df.columns, rf.feature_importances_):
    f_[feature] = importance
f_

 There are : 0.06176394 Fentanyl, 0.08998656 NDrugs, 0.09588853 AnyOpioid , 0.50723653 Age

Let see these data on diagrams:

In [ ]:
filter1 = df_source["Age"]>20 
filter2 =  df_source["Age"]<70
df_source.where(filter1 & filter2, inplace = True)

In [ ]:
sns.relplot(x="Age", y="NDrugs",hue = 'Death', kind="line", data=df_source[['Death','Age','NDrugs']]);

Amount of drugs directly affects mortality

In [ ]:
sns.relplot(x="Age", y="Fentanyl",hue = 'Death', kind="line", data=df_source[['Death','Age','Fentanyl']]);

In [ ]:
sns.relplot(x="Age", y="Heroin",hue = 'Death', kind="line", data=df_source[['Death','Age','Heroin']]);

In [ ]:
sns.relplot(x="Age", y="AnyOpioid",hue = 'Death', kind="line", data=df_source[['Death','Age','AnyOpioid']]);

Fentanyl = most dangerous drug

Let's test our algorithm and enter test data reflecting our conclusion that Age, quantity, and certain types of drugs are the most dangerous.

In [ ]:
df.info()

In [ ]:
rf.predict([[30,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,3]])

In [ ]:
rf.predict([[65,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,4]])

As we can see, the algorithm worked in accordance with our conclusions. The first test set-did not lead to mortality, the second led to